In [2]:
# --- Importaciones ---
import os
import time
import google.generativeai as genai
from dotenv import load_dotenv

# --- Constantes ---
MAX_RETRIES = 3
MAX_TOTAL_TOKENS = 32000        # Límite total del modelo
RESERVED_OUTPUT = 8000          # Tokens reservados inicialmente para la respuesta
SUMMARY_MAX_TOKENS = 500         # Tokens para resumen del historial
SUMMARY_FILE = "conversation_summary.txt"

# --- Funciones de carga ---
def load_prompt_from_file(file_path: str, is_critical: bool = True) -> str | None:
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            return file.read()
    except FileNotFoundError:
        if is_critical:
            print(f"Error: El archivo '{file_path}' no se encontró. No se puede continuar.")
            return None
        print(f"Advertencia: El archivo '{file_path}' no se encontró. Usando contenido vacío.")
        return ""

# --- Funciones de historial ---
def save_summary_to_file(summary: str):
    with open(SUMMARY_FILE, "a", encoding="utf-8") as f:
        f.write("\n--- NUEVO RESUMEN ---\n")
        f.write(summary)
        f.write("\n")

def summarize_history(model, history):
    print("⚡ Resumiendo historial para liberar tokens...")
    summarizer = model.start_chat(history=[])
    response = summarizer.send_message(
        "Resume la siguiente conversación en menos de 500 palabras, manteniendo la información clave:\n\n" + str(history)
    )
    summary_text = response.text or "Resumen no disponible."
    save_summary_to_file(summary_text)
    return summary_text

def ensure_history_fits(chat, model, new_message, max_total_tokens=MAX_TOTAL_TOKENS, reserved_output=RESERVED_OUTPUT):
    # Simular historial con nuevo mensaje
    simulated_history = chat.history + [{"role": "user", "parts": [new_message]}]
    tokens_input = model.count_tokens(simulated_history).total_tokens

    if tokens_input + reserved_output > max_total_tokens:
        # Resumimos la mitad más antigua del historial
        old_messages = chat.history[: len(chat.history)//2]
        recent_messages = chat.history[len(chat.history)//2:]

        summary_text = summarize_history(model, old_messages)

        # Reemplazamos historial viejo con resumen
        chat.history = [{"role": "system", "parts": [f"Resumen de la conversación anterior:\n{summary_text}"]}]
        chat.history.extend(recent_messages)

    # Retornar tokens de entrada calculados con historial ajustado
    tokens_input = model.count_tokens(chat.history + [{"role": "user", "parts": [new_message]}]).total_tokens
    return chat.history, tokens_input

# --- Función principal ---
def main():
    load_dotenv()

    # Archivos de configuración
    prompt_file = "firefly_prompt_user_tko.txt"
    user_history_file = "firefly_initial_history_user_tko.txt"
    model_history_file = "firefly_initial_history_model_tko.txt"

    # Cargar instrucciones del sistema
    system_instruction = load_prompt_from_file(prompt_file, is_critical=True)
    if system_instruction is None:
        return

    # Configurar API de Gemini
    api_key = os.getenv("GEMINI_API_KEY")
    if not api_key:
        print("Error: No se encontró la clave de API en .env")
        return
    genai.configure(api_key=api_key)
    print("Gemini API configured successfully.")

    # Cargar historial previo
    user_hist = load_prompt_from_file(user_history_file, is_critical=False)
    model_hist = load_prompt_from_file(model_history_file, is_critical=False)

    initial_history = []
    if user_hist:
        initial_history.append({"role": "user", "parts": [user_hist]})

    initial_history.extend([
        {"role": "user", "parts": ["Por favor, confirma que estás listo para recibir mis preguntas sobre generación de prompts para Suno."]},
        {"role": "model", "parts": ["Información procesada. Estoy listo para generar tus prompts para Suno."]}
    ])

    # Inicializar chat
    model = genai.GenerativeModel(
        "gemini-2.5-pro",
        system_instruction=system_instruction
    )
    chat = model.start_chat(history=initial_history)

    print("\nEnter your prompt. Type 'exit' to finish.")

    while True:
        user_input = input("Tú: ")
        if user_input.lower() == "exit":
            print("Gemini: Good bye!")
            break

        # Ajustar historial y calcular tokens de entrada
        chat.history, tokens_input = ensure_history_fits(chat, model, user_input)

        # Ajustar dinámicamente el límite de salida
        max_output_tokens = min(RESERVED_OUTPUT, MAX_TOTAL_TOKENS - tokens_input)

        for attempt in range(MAX_RETRIES):
            try:
                print("Gemini: wait")
                response = chat.send_message(user_input, generation_config={"max_output_tokens": max_output_tokens})

                if response.text:
                    print("Gemini:", response.text)

                    # Tokens de salida
                    tokens_output = model.count_tokens([{"role": "model", "parts": [response.text]}]).total_tokens
                    total_tokens = tokens_input + tokens_output

                    # Mostrar desglose
                    print(f"[Input tokens: {tokens_input}]")
                    print(f"[Output tokens: {tokens_output}]")
                    print(f"[Total tokens: {total_tokens}]")
                    break

                else:
                    print("\nGemini: No puedo generar respuesta por políticas de seguridad.")
                    break

            except Exception as e:
                if "500 An internal error" in str(e) and attempt < MAX_RETRIES - 1:
                    print(f"Error 500. Reintentando en 5 segundos... ({attempt+1}/{MAX_RETRIES})")
                    time.sleep(5)
                else:
                    print(f"Error inesperado al comunicarse con la API: {e}")
                    break

        else:
            print("Gemini: No se pudo establecer conexión después de varios intentos.")


if __name__ == "__main__":
    main()


Gemini API configured successfully.

Enter your prompt. Type 'exit' to finish.


Tú:  make a prompt of a cat under the table


Gemini: wait
Gemini: Prompt: A fluffy ginger tabby cat with bright green eyes peeking curiously from the shadows under a rustic wooden dining table, soft focus on the background, hyper-detailed fur, cinematic.
Mood: Cozy, curious, domestic, quiet
TextInImage:
AspectRatio: 16:9
ContentType: Photo
Model: Image4Ultra
Reference:
EffectsStyles: Bokeh, High Detail
ColorTone: Warm tones, golden
Lighting: Low key lighting, a single beam of afternoon sunlight hitting the floor, casting long shadows
CameraAngle: Low angle shot, eye-level with the cat, shallow depth of field
[Input tokens: 289]
[Output tokens: 169]
[Total tokens: 458]


Tú:  exit


Gemini: Good bye!
